In [1]:
import tensorflow as tf
from tensorflow.keras import Input, Sequential, Model
from tensorflow.keras.layers import Dense

In [81]:
NLOOPS=6

In [71]:
N_loops = NLOOPS

# Static looping - vanilla list

## Comparing Add layer vs lambda op

In [72]:
tf.random.set_seed(1)

_in = Input((5,), name="vanilla_list_static")
x = _in
lst = []
# static looping
for i in range(N_loops):
    y = Dense(2, name=f"{i}")(x)
    for j, z in enumerate(lst):
        # y = tf.keras.layers.Add(name=f"add_{i}_{j}")([y, z])
        y = tf.math.add(y,z,name=f"add_{i}_{j}")
    lst.append(y)
    x = y
model = Model(_in, x)
model.build((3,))
model.summary()
# tf.keras.utils.plot_model(model, expand_nested=True)

Model: "model_30"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
vanilla_list_static (InputLayer [(None, 5)]          0                                            
__________________________________________________________________________________________________
0 (Dense)                       (None, 2)            12          vanilla_list_static[0][0]        
__________________________________________________________________________________________________
1 (Dense)                       (None, 2)            6           0[0][0]                          
__________________________________________________________________________________________________
tf.math.add_640 (TFOpLambda)    (None, 2)            0           1[0][0]                          
                                                                 0[0][0]                   

In [73]:
%%timeit -r 20
model(tf.ones((1,5)))

3.85 ms ± 221 µs per loop (mean ± std. dev. of 20 runs, 100 loops each)


In [74]:
tf.random.set_seed(1)

_in = Input((5,), name="vanilla_list_static")
x = _in
lst = []
# dynamic looping
for i in range(N_loops):
    y = Dense(2, name=f"{i}")(x)
    for j, z in enumerate(lst):
        y = tf.keras.layers.Add(name=f"add_{i}_{j}")([y, z])
        # y = tf.math.add(y,z,name=f"add_{i}_{j}")
    lst.append(y)
    x = y
model = Model(_in, x)
model.build((3,))

model.summary()
# tf.keras.utils.plot_model(model, expand_nested=True)

Model: "model_31"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
vanilla_list_static (InputLayer [(None, 5)]          0                                            
__________________________________________________________________________________________________
0 (Dense)                       (None, 2)            12          vanilla_list_static[0][0]        
__________________________________________________________________________________________________
1 (Dense)                       (None, 2)            6           0[0][0]                          
__________________________________________________________________________________________________
add_1_0 (Add)                   (None, 2)            0           1[0][0]                          
                                                                 0[0][0]                   

In [75]:
%%timeit -r 20
model(tf.ones((1,5)))

4.15 ms ± 792 µs per loop (mean ± std. dev. of 20 runs, 100 loops each)


# Dynamic Looping - vanilla list

Seems to be broken - thus suggesting that Python is ok as the vehicle for building graphs.

Unclear _when_ we would want to use tensorarrays but I figure it is for implementing certain call methods?

In [82]:
N_loops = NLOOPS

In [83]:
tf.random.set_seed(1)

@tf.function
def gen_model():
    _in = Input((5,), name="vanilla_list_static")
    x = _in
    lst = tf.TensorArray(_in.dtype, N_loops, 
                        dynamic_size=False, 
                        clear_after_read=False, 
                        tensor_array_name="lst", 
                        element_shape=tf.TensorShape((3,))
                        )
    max_j = 0
    # static looping
    for i in range(N_loops):
        y = Dense(2, name=f"{i}")(x)
        for j in range(max_j):
            y = tf.keras.layers.Add(name=f"add_{i}_{j}")([y, lst.read(j)])
            # y = tf.math.add(y,z,name=f"add_{i}_{j}")
        lst = lst.write(i, y)
        x = y
        max_j += 1
    model = Model(_in, x)
    return model
model = gen_model()
model.build((3,))

model.summary()
# tf.keras.utils.plot_model(model, expand_nested=True)

TypeError: in user code:

    /tmp/ipykernel_367763/3522315067.py:20 gen_model  *
        lst = lst.write(i, y)
    /home/alexander/projects/deep_learning_vs_gbdt/venv/lib/python3.9/site-packages/tensorflow/python/util/tf_should_use.py:247 wrapped  **
        return _add_should_use_warning(fn(*args, **kwargs),
    /home/alexander/projects/deep_learning_vs_gbdt/venv/lib/python3.9/site-packages/tensorflow/python/ops/tensor_array_ops.py:1156 write
        return self._implementation.write(index, value, name=name)
    /home/alexander/projects/deep_learning_vs_gbdt/venv/lib/python3.9/site-packages/tensorflow/python/ops/tensor_array_ops.py:534 write
        value = ops.convert_to_tensor(
    /home/alexander/projects/deep_learning_vs_gbdt/venv/lib/python3.9/site-packages/tensorflow/python/profiler/trace.py:163 wrapped
        return func(*args, **kwargs)
    /home/alexander/projects/deep_learning_vs_gbdt/venv/lib/python3.9/site-packages/tensorflow/python/framework/ops.py:1566 convert_to_tenso

In [101]:
class TensorArrayModel(Model):
    def __init__(self, N_loops, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.dense_layers = [Dense(2, name=f"{i}") for i in range(N_loops)]
        self.n_loops = N_loops
    
    @tf.function
    def call(self, inputs, training=None, mask=None):
        units = 2
        lst = tf.TensorArray(_in.dtype, self.n_loops, 
                            dynamic_size=False, 
                            clear_after_read=False, 
                            tensor_array_name="lst", 
                            element_shape=tf.TensorShape((None,units))
                            )
        max_j = 0
        x = inputs
        # static looping
        for i in range(N_loops):
            y = self.dense_layers[i](x)
            for j in range(max_j):
                y = tf.keras.layers.Add(name=f"add_{i}_{j}")([y, lst.read(j)])
                # y = tf.math.add(y,z,name=f"add_{i}_{j}")
            lst = lst.write(i, y)
            x = y
            max_j += 1
        # lst.marked_used()
        return x

tf.random.set_seed(1)
model = TensorArrayModel(N_loops=NLOOPS)

In [102]:
%%timeit -r 20
model(tf.ones((1,5)))

777 µs ± 216 µs per loop (mean ± std. dev. of 20 runs, 1 loop each)


# Static Looping - tensorarray

# Dynamic Looping - tensorarray